# Table of Contents
* [Comparison of Hi-C experiments](#Comparison-of-Hi-C-experiments)
	* [Between conditions](#Between-conditions)
		* [HindIII](#HindIII)
		* [NcoI](#NcoI)
	* [Between replicates with different restriction enzymes](#Between-replicates-with-different-restriction-enzymes)
		* [T0](#T0)
		* [T60](#T60)
* [Merge Hi-C experiments](#Merge-Hi-C-experiments)
	* [Normalizing merged data](#Normalizing-merged-data)


# Comparison of Hi-C experiments

## Some settings

In [1]:
from pytadbit.mapping.analyze import eig_correlate_matrices, correlate_matrices
from pytadbit import load_hic_data_from_reads
from cPickle import load
from matplotlib import pyplot as plt

In [2]:
reso = 1000000
base_path = 'results/{0}/03_filtering/valid_reads12_{0}.tsv'
bias_ice_path = 'results/{1}/04_normalizing/biases_{0}_{1}.pick'
bias_dry_path = 'results/{1}/04_normalizing/biases_dryhic_{0}_{1}.tsv'
bads_path = 'results/{1}/04_normalizing/bad_columns_{0}_{1}.pick'

Write a little function to load HiCData obeject

In [3]:
def my_load_hic_data(renz, reso, which='ice'):
    hic_data = load_hic_data_from_reads(base_path.format(renz), resolution=reso)
    if which=='ice':
        hic_data.bias = load(open(bias_ice_path.format(reso, renz)))
    else:
        hic_data.bias = dict([(int(l.split()[0]), float(l.split()[1])) 
                              for l in open(bias_dry_path.format(reso, renz))])
    hic_data.bads = load(open(bads_path.format(reso, renz)))
    return hic_data

In [4]:
renz1 = 'HindIII'
renz2 = 'MboI'

## Load data

### ICE normalized

In [5]:
hic_data1_ice = my_load_hic_data(renz1, reso, which='ice')
hic_data2_ice = my_load_hic_data(renz2, reso, which='ice')

### DRY normalized

In [6]:
hic_data1_dry = my_load_hic_data(renz1, reso, which='dry')
hic_data2_dry = my_load_hic_data(renz2, reso, which='dry')

In [ ]:
%matplotlib inline

## Plot correlations

### ICE

In [ ]:
## this part is to "tune" the plot ##
plt.figure(figsize=(9, 6))
axe = plt.subplot()
axe.grid()
axe.set_xticks(range(0, 55, 5))
axe.set_xticklabels(['%d Mb' % int(i * 0.2) if i else '' for i in range(0, 55, 5)], rotation=-45)
#####################################

_ = correlate_matrices(hic_data1_ice, hic_data2_ice, max_dist=50, show=False, axe=axe, normalized=True)

### DRY

In [ ]:
## this part is to "tune" the plot ##
plt.figure(figsize=(9, 6))
axe = plt.subplot()
axe.grid()
axe.set_xticks(range(0, 55, 5))
axe.set_xticklabels(['%d Mb' % int(i * 0.2) if i else '' for i in range(0, 55, 5)], rotation=-45)
#####################################

_ = correlate_matrices(hic_data1_dry, hic_data2_dry, max_dist=50, show=False, axe=axe, normalized=True)

## Repeat at 1 Mb resolution


In [ ]:
reso = 1000000
hic_data1_ice = my_load_hic_data(renz1, reso, which='ice')
hic_data2_ice = my_load_hic_data(renz2, reso, which='ice')
hic_data1_dry = my_load_hic_data(renz1, reso, which='dry')
hic_data2_dry = my_load_hic_data(renz2, reso, which='dry')

### ICE

In [ ]:
## this part is to "tune" the plot ##
plt.figure(figsize=(9, 6))
axe = plt.subplot()
axe.grid()
axe.set_xticks(range(0, 55, 5))
axe.set_xticklabels(['%d Mb' % int(i * 0.2) if i else '' for i in range(0, 55, 5)], rotation=-45)
#####################################

_ = correlate_matrices(hic_data1_ice, hic_data2_ice, max_dist=50, show=False, axe=axe, normalized=True)

### DRY

In [ ]:
## this part is to "tune" the plot ##
plt.figure(figsize=(9, 6))
axe = plt.subplot()
axe.grid()
axe.set_xticks(range(0, 55, 5))
axe.set_xticklabels(['%d Mb' % int(i * 0.2) if i else '' for i in range(0, 55, 5)], rotation=-45)
#####################################

_ = correlate_matrices(hic_data1_dry, hic_data2_dry, max_dist=50, show=False, axe=axe, normalized=True)

## Compare eigenvectors

### ICE

In [ ]:
corrs = eig_correlate_matrices(hic_data1_ice, hic_data2_ice, show=True, aspect='auto', normalized=True)

for cor in corrs:
    print ' '.join(['%5.3f' % (c) for c in cor]) + '\n'

### DRY

In [ ]:
hic_data1_dry.bias = [for b in hic_data1_dry.bias]

In [ ]:
corrs = eig_correlate_matrices(hic_data1_dry, hic_data2_dry, show=True, aspect='auto', normalized=True)

for cor in corrs:
    print ' '.join(['%5.3f' % (c) for c in cor]) + '\n'

# Merge Hi-C experiments

Once agreed that experiments are similar, they can be merged.

Here is a simple way to merge valid pairs. Arguably we may want to merge unfiltered data but the difference would be minimal specially with non-replicates.

In [ ]:
from pytadbit.mapping import merge_2d_beds

In [ ]:
! mkdir -p results/fragment/both_T0/
! mkdir -p results/fragment/both_T60/
! mkdir -p results/fragment/both_T0/03_filtering/
! mkdir -p results/fragment/both_T60/03_filtering/

In [ ]:
renz1 = 'HindIII'
renz2 = 'NcoI'
rep   = 'T60'

hic_data1 = 'results/fragment/{0}_{1}/03_filtering/valid_reads12_{0}_{1}.tsv'.format(renz1, rep)
hic_data2 = 'results/fragment/{0}_{1}/03_filtering/valid_reads12_{0}_{1}.tsv'.format(renz2, rep)
hic_data  = 'results/fragment/both_{0}/03_filtering/valid_reads12_{0}.tsv'.format(rep)

merge_2d_beds(hic_data1, hic_data2, hic_data)

In [ ]:
renz1 = 'HindIII'
renz2 = 'NcoI'
rep   = 'T0'

hic_data1 = 'results/fragment/{0}_{1}/03_filtering/valid_reads12_{0}_{1}.tsv'.format(renz1, rep)
hic_data2 = 'results/fragment/{0}_{1}/03_filtering/valid_reads12_{0}_{1}.tsv'.format(renz2, rep)
hic_data  = 'results/fragment/both_{0}/03_filtering/valid_reads12_{0}.tsv'.format(rep)

merge_2d_beds(hic_data1, hic_data2, hic_data)

## Normalizing merged data

In [ ]:
from pytadbit.mapping.analyze import hic_map
from cPickle import dump

In [ ]:
! mkdir -p results/fragment/both_T0/04_normalizing
! mkdir -p results/fragment/both_T60/04_normalizing

All in one loop to:
 - filter
 - normalize
 - generate intra-chromosome and genomic matrices

all at diferent resolutions, and for both time points

In [ ]:
for rep in ['T0', 'T60']:
    print ' -', rep
    for reso in [1000000, 300000, 100000]:
        print '   *', reso
        # load hic_data
        hic_data = load_hic_data_from_reads(
            'results/fragment/both_{0}/03_filtering/valid_reads12_{0}.tsv'.format(rep),
            reso)
        # filter columns
        hic_data.filter_columns(draw_hist=False, min_count=10, by_mean=True)
        # normalize
        hic_data.normalize_hic(iterations=0)
        # save biases to reconstruct normalization
        out = open('results/fragment/both_{1}/04_normalizing/biases_{0}_{1}.pick'.format(reso, rep), 'w')
        dump(hic_data.bias, out)
        out.close()
        # save filtered out columns
        out = open('results/fragment/both_{1}/04_normalizing/bad_columns_{0}_{1}.pick'.format(reso, rep), 'w')
        dump(hic_data.bads, out)
        out.close()
        # save data as raw matrix per chromsome
        hic_map(hic_data, by_chrom='intra', normalized=False,
                savedata='results/fragment/both_{1}/04_normalizing/{0}_raw'.format(reso, rep))
        # save data as normalized matrix per chromosome
        hic_map(hic_data, by_chrom='intra', normalized=True,
                savedata='results/fragment/both_{1}/04_normalizing/{0}_norm'.format(reso, rep))
        # if the resolution is low save the full genomic matrix
        if reso > 500000:
            hic_map(hic_data, by_chrom=False, normalized=False, 
                    savefig ='results/fragment/both_{1}/04_normalizing/{0}_raw.png'.format(reso, rep),
                    savedata='results/fragment/both_{1}/04_normalizing/{0}_raw.mat'.format(reso, rep))

            hic_map(hic_data, by_chrom=False, normalized=True,
                    savefig ='results/fragment/both_{1}/04_normalizing/{0}_norm.png'.format(reso, rep) ,
                    savedata='results/fragment/both_{1}/04_normalizing/{0}_norm.mat'.format(reso, rep))